In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import dirname, join
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
season_path1 = "Documents/AML Final Project/data/2016-17/gws/merged_gw.csv"
season_path2 = "Documents/AML Final Project/data/2017-18/gws/merged_gw.csv"
season_path3 = "Documents/AML Final Project/data/2018-19/gws/merged_gw.csv"
season_path4 = "Documents/AML Final Project/data/2019-20/gws/merged_gw.csv"
season_path5 = "Documents/AML Final Project/data/2020-21/gws/merged_gw.csv"
season_path6 = "Documents/AML Final Project/data/2021-22/gws/merged_gw.csv"
season_path7 = "Documents/AML Final Project/data/2022-23/gws/merged_gw.csv"
season_path8 = "Documents/AML Final Project/data/2023-24/gws/merged_gw.csv"

In [3]:
required_columns = [
    'season', 'name', 'position', 'team', 'assists', 'bonus', 'bps', 
    'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded', 
    'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes', 
    'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved', 
    'red_cards', 'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 
    'threat', 'total_points', 'transfers_balance', 'transfers_in', 
    'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW'
]

In [4]:
def read_and_standardize(file_path, encoding, season):
    df = pd.read_csv(file_path, encoding=encoding)    
    missing_columns = [col for col in required_columns if col not in df.columns]
    for col in missing_columns:
        df[col] = np.nan
    df['season'] = season
    return df[required_columns]

# Reading and standardizing datasets
data1 = read_and_standardize(season_path1, 'ISO-8859-1', "2016-17")
data2 = read_and_standardize(season_path2, 'ISO-8859-1', "2017-18")
data3 = read_and_standardize(season_path3, 'ISO-8859-1', "2018-19")
data4 = read_and_standardize(season_path4, 'utf-8', "2019-20")
data5 = read_and_standardize(season_path5, 'utf-8', "2020-21")
data6 = read_and_standardize(season_path6, 'utf-8', "2021-22")
data7 = read_and_standardize(season_path7, 'utf-8', "2022-23")
data8 = read_and_standardize(season_path8, 'utf-8', "2023-24")

In [5]:
# data8['position'].unique()[:20]
data8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29725 entries, 0 to 29724
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season             29725 non-null  object 
 1   name               29725 non-null  object 
 2   position           29725 non-null  object 
 3   team               29725 non-null  object 
 4   assists            29725 non-null  int64  
 5   bonus              29725 non-null  int64  
 6   bps                29725 non-null  int64  
 7   clean_sheets       29725 non-null  int64  
 8   creativity         29725 non-null  float64
 9   element            29725 non-null  int64  
 10  fixture            29725 non-null  int64  
 11  goals_conceded     29725 non-null  int64  
 12  goals_scored       29725 non-null  int64  
 13  ict_index          29725 non-null  float64
 14  influence          29725 non-null  float64
 15  kickoff_time       29725 non-null  object 
 16  minutes            297

In [6]:
def clean_players_name_string(df, col='name'):
    """ Clean the imported file 'name' column because it has different patterns between seasons

    Args:
        df: merged df for all the seasons that have been imported
        col: name of the column for cleanup
    """
    #replace _ with space in name column
    df[col] = df[col].str.replace('_', ' ', regex=False)
    #remove number in name column
    df[col] = df[col].str.replace(r'\d+', '', regex=False)
    #trim name column
    df[col] = df[col].str.strip()
    df[col] = df[col].str.replace(r'\s+\d+$', '', regex=True)
    return df

In [7]:
df_names = [data1, data2, data3, data4, data5, data6, data7, data8]

column_types = data8.dtypes
for i in range(len(df_names)):
    for col in df_names[i].columns:
        if col in column_types:
            # Convert column type to match the corresponding column in data8
            if column_types[col] == 'int64':  # Check if the column type is int
                df_names[i][col] = df_names[i][col].astype('Int64')  # Convert to nullable Int64
            else:
                df_names[i][col] = df_names[i][col].astype(column_types[col])

for i in df_names:
    i = clean_players_name_string(i, col='name')

In [8]:
# data1.head(20)
# data5.info()
# data5['name'].head(20)
data1['position'].unique()[:20]

# for df in df_names:
#     print(df['assists'].info())  # Check the first few rows of 'position' column in each DataFrame

array([nan], dtype=object)

In [9]:
df_master_team = pd.read_csv("Documents/AML Final Project/data/master_team_list.csv")

In [10]:
def assign_team_names_prev(merged_gw, master_team_list):
    def process_fixture(group):
        # Get distinct opponent teams for this fixture
        unique_teams = group['opponent_team'].unique()
        if len(unique_teams) != 2:
            raise ValueError(f"Fixture {group['fixture'].iloc[0]} does not have exactly 2 teams.")
        
        # Filter master_team_list based on season from merged_gw
        master_team_season = master_team_list[master_team_list['season'] == group['season'].iloc[0]]
        
        # Map opponent team names based on the filtered master_team_list
        team_mapping = master_team_season.set_index('team')['team_name'].to_dict()
        team_1_name = team_mapping[unique_teams[0]]
        team_2_name = team_mapping[unique_teams[1]]
        
        # Update 'team' and 'opponent_team' columns based on the team names
        group['team'] = group['opponent_team'].map({
            unique_teams[0]: team_2_name,
            unique_teams[1]: team_1_name
        })
        
        group['opponent_team'] = group['opponent_team'].map({
            unique_teams[0]: team_1_name,
            unique_teams[1]: team_2_name
        })
        return group
    
    # Group by fixture and season, and process each group
    return merged_gw.groupby(['fixture', 'season'], group_keys=False).apply(process_fixture).reset_index(drop=True)

In [11]:
for i in range(len(df_names)):
    df_names[i] = assign_team_names_prev(df_names[i], df_master_team)

In [12]:
# data8.info()
for i, df in enumerate(df_names):
    print(f"Shape of dataframe {i+1}: {df.shape}")

print(f"Shape of concatenated dataframe: {df.shape}")

Shape of dataframe 1: (23679, 36)
Shape of dataframe 2: (22467, 36)
Shape of dataframe 3: (21790, 36)
Shape of dataframe 4: (22560, 36)
Shape of dataframe 5: (24365, 36)
Shape of dataframe 6: (25447, 36)
Shape of dataframe 7: (26505, 36)
Shape of dataframe 8: (29725, 36)
Shape of concatenated dataframe: (29725, 36)


In [13]:
# Concatenate all dataframes
raw_df = pd.concat(df_names, ignore_index=True, sort=False)

In [14]:
# raw_df['position'].unique()[:20]
raw_df[raw_df['season'] == '2016-17']['position'].unique()[:20]

array([nan], dtype=object)

In [15]:
# df_filtered = raw_df[raw_df['season'] == '2023-24']
# print(df_filtered.shape)  # Check the resulting shape
# print(df_filtered.head(5))  # Inspect the first few rows

In [16]:
raw_df['season'].unique()

array(['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23', '2023-24'], dtype=object)

In [17]:
def filter_players_exist_latest(df, col='position', season_col='season', latest_season='2023-24'):
    """ Fill in null 'position' (data that only available in 20-21 season) into previous seasons. 
        Null meaning that player doesnt exist in latest season hence can exclude.
    """
    pd.set_option('future.no_silent_downcasting', True)
    result = df.groupby('name')[col].apply(lambda x: x.ffill().bfill())
    df[col] = result.droplevel(0)
    # df = df[df[col].notnull()]
    
    return df

In [18]:
latest_df = filter_players_exist_latest(raw_df, col='position', season_col='season', latest_season='2023-24') 

In [19]:
latest_df['season'].unique()

array(['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23', '2023-24'], dtype=object)

In [20]:
# df_filtered = latest_df[latest_df['season'] == '2018-19']
# print(df_filtered.shape)  # Check the resulting shape
# print(df_filtered.head())  # Inspect the first few rows

In [21]:
# Group by name and season, summing the minutes column
minutes_sum = latest_df.groupby(['name', 'season'])['minutes'].sum().reset_index()
# print(minutes_sum)

# Identify players with zero minutes for a specific season
players_zero_minutes_season = minutes_sum[minutes_sum['minutes'] == 0][['name', 'season']]
# print(players_zero_minutes_season.head(50))

# Merge with the main dataframe and exclude rows for players with zero minutes in specific seasons
filtered_latest_df = latest_df.merge(
    players_zero_minutes_season,
    on=['name', 'season'],
    how='left',
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge'])

# Display the updated dataframe
# print(filtered_latest_df)

In [22]:
def export_cleaned_data(df, filename):
    """ Function to export merged df into specified folder
    Args:path (str): Path of the folder, filename(str): Name of the file """
    filename = filename
    filepath = join(dirname(dirname("__file__")), 'Documents/AML Final Project/data', filename)
    df.to_csv(filepath, encoding = 'utf-8', index=False)
    return df

In [23]:
export_cleaned_data(filtered_latest_df, 'cleaned_merged_seasons_updated.csv')

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,West Ham,0,0,0,0,0.0,454,...,2,0.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,MID,Everton,0,0,6,0,0.3,142,...,1,0.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,Arsenal,0,0,5,0,4.9,16,...,3,23.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,Watford,0,0,0,0,0.0,482,...,1,0.0,0,0,0,0,50,False,0,1
5,2016-17,Abel Hernández,NaN,Hull,1,0,10,0,12.2,163,...,2,30.0,5,0,0,0,60,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196529,2023-24,Andros Townsend,MID,Luton,0,0,3,0,0.1,727,...,2,0.0,1,-85,152,237,50,True,0,38
196530,2023-24,Chris Richards,DEF,Crystal Palace,0,0,29,1,11.4,238,...,5,0.0,6,19929,25020,5091,39,True,0,38
196532,2023-24,Daniel Gore,MID,Man Utd,0,0,0,0,0.0,685,...,0,0.0,0,-3,0,3,45,False,0,38
196533,2023-24,Kyle Walker,DEF,Man City,0,0,15,0,11.6,369,...,3,0.0,2,-3560,44071,47631,55,True,0,38


In [24]:
# Filter the DataFrame to keep only the required columns
fixture_df = filtered_latest_df[['season','GW', 'team', 'opponent_team', 'fixture', 'kickoff_time', 'round', 'team_a_score', 'team_h_score', 'was_home']]

# Drop duplicate rows based on these columns
fixture_df = fixture_df.drop_duplicates()
df_fixture = fixture_df.sort_values(by=['season', 'GW', 'fixture'])

# Calculate scores for each team and opponent based on the 'was_home' column
df_fixture['team_score'] = df_fixture.apply(lambda row: row['team_h_score'] if row['was_home'] else row['team_a_score'], axis=1)
df_fixture['opponent_score'] = df_fixture.apply(lambda row: row['team_a_score'] if row['was_home'] else row['team_h_score'], axis=1)

# Add 'Goal Difference' column by subtracting 'opponent_score' from 'team_score'
df_fixture['Goal Difference'] = df_fixture['team_score'] - df_fixture['opponent_score']

# Drop columns: 'team_a_score', 'team_h_score', 'kickoff_time'
df_fixture = df_fixture.drop(columns=['team_a_score', 'team_h_score', 'kickoff_time'])

# Convert float columns to int (if needed)
float_columns = df_fixture.select_dtypes(include=['float']).columns
df_fixture[float_columns] = df_fixture[float_columns].astype('Int64')

export_cleaned_data(df_fixture, 'cleaned_fixtures_final.csv')

    season                    name position           team  assists  bonus  \
0  2023-24             Femi Seriki      DEF  Sheffield Utd        0      0   
1  2023-24        Jack Hinshelwood      MID       Brighton        0      0   
2  2023-24            Jadon Sancho      MID        Man Utd        0      0   
3  2023-24  Rhys Norrington-Davies      DEF  Sheffield Utd        0      0   
4  2023-24           Vitaly Janelt      MID      Brentford        0      0   
5  2023-24              Ionuț Radu       GK    Bournemouth        0      0   
6  2023-24            Andre Brooks      MID  Sheffield Utd        0      0   
7  2023-24           Jack Grealish      MID       Man City        0      0   
8  2023-24             Emil Krafth      DEF      Newcastle        0      0   
9  2023-24         Boubacar Traoré      MID         Wolves        0      0   

   bps  clean_sheets  creativity  element  ...  team_h_score  threat  \
0    0             0         0.0      653  ...             0     0.0 

,season,GW,team,opponent_team,fixture,round,was_home,team_score,opponent_score,Goal Difference
37,2016-17,1,Burnley,Swansea,1,1,True,0,1,-1
43,2016-17,1,Swansea,Burnley,1,1,False,1,0,1
38,2016-17,1,Crystal Palace,West Brom,2,1,True,0,1,-1
63,2016-17,1,West Brom,Crystal Palace,2,1,False,1,0,1
1,2016-17,1,Everton,Spurs,3,1,True,1,1,0
...,...,...,...,...,...,...,...,...,...,...
195750,2023-24,38,Fulham,Luton,378,38,False,4,2,2
195684,2023-24,38,Man City,West Ham,379,38,True,3,1,2
195702,2023-24,38,West Ham,Man City,379,38,False,1,3,-2
195679,2023-24,38,Sheffield Utd,Spurs,380,38,True,0,3,-3
